In [1]:
import sqlite3 #database
import json #loadin lines from datadump
from datetime import datetime #logging

In [35]:
timeframe= "2015-01"
start_row = 0
cleanup = 1000000
sql_transaction=[]
connection=sqlite3.connect('{}.db'.format (timeframe))
c=connection.cursor()


In [19]:
def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)")

In [42]:
if __name__ == '__main__':
    create_table()
    row_counter = 0
    paired_rows = 0

                
    with open('/Users/darshandoshi/Desktop/git-repositiories/CHATBOT/RC_2015-01', buffering=1000) as f:
        for row in f:
            row_counter += 1
            if row_counter > start_row:
                try:
                    row = json.loads(row)
                    parent_id = row['parent_id'].split('_')[1]
                    body = format_data(row['body'])
                    created_utc = row['created_utc']
                    score = row['score']
                    
                    comment_id = row['id']
                    
                    subreddit = row['subreddit']
                    parent_data = find_parent(parent_id)
                    
                    existing_comment_score = find_existing_score(parent_id)
                    if existing_comment_score:
                        if score > existing_comment_score:
                            if accepted(body):
                                sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                
                    else:
                        if accepted(body):
                            if parent_data:
                                if score >= 2:
                                    sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                    paired_rows += 1
                            else:
                                sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)
                except Exception as e:
                    print(str(e))
                            
            if row_counter % 100000 == 0:
                print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))

            if row_counter % cleanup == 0:
                print("Cleanin up!")
                sql = "DELETE FROM parent_reply WHERE parent IS NULL"
                c.execute(sql)
                connection.commit()
                c.execute("VACUUM")
                connection.commit()

Total Rows Read: 100000, Paired Rows: 5936, Time: 2018-03-26 00:50:39.495343
Total Rows Read: 200000, Paired Rows: 13449, Time: 2018-03-26 00:52:57.652965
Total Rows Read: 300000, Paired Rows: 21660, Time: 2018-03-26 00:55:11.192649
Total Rows Read: 400000, Paired Rows: 29860, Time: 2018-03-26 00:57:17.069917
Total Rows Read: 500000, Paired Rows: 37870, Time: 2018-03-26 00:59:19.340125
Total Rows Read: 600000, Paired Rows: 45130, Time: 2018-03-26 01:01:18.678202
Total Rows Read: 700000, Paired Rows: 52587, Time: 2018-03-26 01:03:12.568969
Total Rows Read: 800000, Paired Rows: 60809, Time: 2018-03-26 01:05:04.030882
Total Rows Read: 900000, Paired Rows: 69269, Time: 2018-03-26 01:06:56.475027
Total Rows Read: 1000000, Paired Rows: 77622, Time: 2018-03-26 01:08:49.746317
Cleanin up!
Total Rows Read: 1100000, Paired Rows: 84543, Time: 2018-03-26 01:11:27.429668
Total Rows Read: 1200000, Paired Rows: 92429, Time: 2018-03-26 01:13:34.059040
Total Rows Read: 1300000, Paired Rows: 100672, Tim

KeyboardInterrupt: 

In [10]:
def format_data(data):
    data = data.replace('\n',' newlinechar ').replace('\r',' newlinechar ').replace('"',"'")
    return data

In [12]:
def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False


In [13]:
def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

In [30]:
def accepted(data):
    if (len(data.split(' '))) > 50 and len(data)<1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]':
        return False
    elif data==['removed']:
        return False
    else:
        return True

In [15]:
def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql="""UPDATE parent_reply set parent_id=? , comment_id=?, parent=?, comment=?, subreddit=?, unix=?, score=? WHERE parent_id=?""" .format(parentid,commentid,parent,comment,subreddit,time,score)
        transaction_bldr(sql)
    except Exception as e:
        print("a0",str(e))

In [16]:
def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql="""INSERT into parent_reply (parent_id,comment_id,parent,comment,subreddit,unix,score) VALUES ("{}","{}","{}","{}","{}","{}","{}")""".format(parentid,commentid,parent,comment,subreddit,time,score)
        transaction_bldr(sql)
    except Exception as e:
        print("a0",str(e))

In [41]:
def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql="""INSERT into parent_reply (parent_id,comment_id,comment,subreddit,unix,score) VALUES ("{}","{}","{}","{}","{}","{}")""".format(parentid,commentid,comment,subreddit,time,score)
        transaction_bldr(sql)
    except Exception as e:
        print("a1",str(e))

In [21]:
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c. execute("BEGIN TRANSACTION")
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
            connection.commit()
            sql_transaction=[]